# Inspection of Gazebo models

## List all Gazebo models

The library loads all Gazebo models in the 

* catkin workspace
* `$HOME/.gazebo/models`
* `/usr/share/gazebo-$GAZEBO_VERSION/models`

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
from pcg_gazebo.simulation import get_gazebo_model_names, add_custom_gazebo_resource_path
from pcg_gazebo.parsers.sdf import create_sdf_element

# Add the local Gazebo models resources folder
add_custom_gazebo_resource_path(os.path.abspath('models'))

gazebo_models = list(get_gazebo_model_names())

if len(gazebo_models) == 0:
    print('No models found')
    exit()
    
gazebo_models.remove('sun')
gazebo_models.remove('ground_plane')

print('List of models:')
for tag in gazebo_models:
    print('\t - {}'.format(tag))

List of models:
	 - cardboard_box
	 - arm_part
	 - euro_pallet
	 - walls
	 - speed_limit_sign
	 - mobile_base
	 - OrangeDoors
	 - hinged_door
	 - pcg_jansport_backpack_red
	 - pcg_warehouse_robot


## Loading Gazebo models already in the database

Using the tag for the Gazebo model, the SDF file can be loaded as a `SimulationObject`.

In [2]:
from pcg_gazebo.simulation import SimulationModel
import random

model = SimulationModel.from_gazebo_model('pcg_warehouse_robot')

### Visualize the visual and collision geometries of the model using `pyglet`.

In [3]:
model.show(mesh_type='collision')

SceneViewer(width=1920, height=1052)

![sim_warehouse_robot_collision](images/sim_warehouse_robot_collision.png)

In [5]:
model.show(mesh_type='visual')

SceneViewer(width=1920, height=1052)

![sim_warehouse_robot_visual](images/sim_warehouse_robot_visual.png)

## Inspect the SDF elements

Load another Gazebo model and inspect the SDF elements directly.

In [6]:
# Open a Gazebo model
model_name = random.choice(gazebo_models)
print('Import Gazebo model={}'.format(model_name))

model = SimulationModel.from_gazebo_model('pcg_jansport_backpack_red')
# Visualize its visual geometries
model.show(mesh_type='visual')

Import Gazebo model=arm_part


SceneViewer(width=1920, height=1052)

![sim_backpack_visual](images/sim_backpack_visual.png)

Retrieve the SDF elements using the Python SDF implementations in the `pcg_gazebo.parsers.sdf` elements.

In [7]:
sdf = model.to_sdf()
print(sdf)

<model name="jansport_backpack_red">
  <pose frame="">0 0 0 0 -0 0</pose>
  <static>1</static>
  <self_collide>0</self_collide>
  <allow_auto_disable>1</allow_auto_disable>
  <link name="link">
    <collision name="collision">
      <geometry>
        <mesh>
          <uri>model://pcg_jansport_backpack_red/meshes/backpack.dae</uri>
          <scale>1 1 1</scale>
        </mesh>
      </geometry>
      <pose frame="">0 0 0 0 -0 0</pose>
      <max_contacts>20</max_contacts>
    </collision>
    <visual name="visual">
      <geometry>
        <mesh>
          <uri>model://pcg_jansport_backpack_red/meshes/backpack.dae</uri>
          <scale>1 1 1</scale>
        </mesh>
      </geometry>
      <pose frame="">0 0 0 0 -0 0</pose>
      <cast_shadows>1</cast_shadows>
      <transparency>0.0</transparency>
    </visual>
    <pose frame="">0 0 0 0 -0 0</pose>
  </link>
</model>



In [8]:
print('Links: ', sdf.links)

Links:  [<pcg_gazebo.parsers.sdf.link.Link object at 0x7f70422e4f60>]


In [9]:
# Print individual links
for i in range(len(sdf.links)):
    print('Old link #{}'.format(i))
    print(sdf.links[i])
    
    # Add mass to link
    sdf.links[i].inertial = create_sdf_element('inertial')
    sdf.links[i].inertial.mass = random.random()
    sdf.links[i].inertial.inertia.ixx = random.random()
    sdf.links[i].inertial.inertia.iyy = random.random()
    sdf.links[i].inertial.inertia.izz = random.random()
    
    print('New link #{}'.format(i))
    print(sdf.links[i])

Old link #0
<link name="link">
  <collision name="collision">
    <geometry>
      <mesh>
        <uri>model://pcg_jansport_backpack_red/meshes/backpack.dae</uri>
        <scale>1 1 1</scale>
      </mesh>
    </geometry>
    <pose frame="">0 0 0 0 -0 0</pose>
    <max_contacts>20</max_contacts>
  </collision>
  <visual name="visual">
    <geometry>
      <mesh>
        <uri>model://pcg_jansport_backpack_red/meshes/backpack.dae</uri>
        <scale>1 1 1</scale>
      </mesh>
    </geometry>
    <pose frame="">0 0 0 0 -0 0</pose>
    <cast_shadows>1</cast_shadows>
    <transparency>0.0</transparency>
  </visual>
  <pose frame="">0 0 0 0 -0 0</pose>
</link>

New link #0
<link name="link">
  <collision name="collision">
    <geometry>
      <mesh>
        <uri>model://pcg_jansport_backpack_red/meshes/backpack.dae</uri>
        <scale>1 1 1</scale>
      </mesh>
    </geometry>
    <pose frame="">0 0 0 0 -0 0</pose>
    <max_contacts>20</max_contacts>
  </collision>
  <visual name="visual

The SDF elements can be modified directly and the SDF file can be exported again.

In [10]:
import os

sdf.name = 'new_name'

if not os.path.isdir('sdf'):
    os.makedirs('sdf')
    
sdf.export_xml('/tmp/modified_sdf.sdf')